huggingface-cli install & login  
https://huggingface.co/docs/huggingface_hub/guides/cli

In [1]:
# in terminal env
# huggingface-cli download meta-llama/Meta-Llama-3-8B-Instruct --local-dir Meta-Llama-3-8B-Instruct

In [2]:
# transformer 4.42(lateset version to use llama3)
# !pip install transformers
# !pip install torch

In [1]:
!nvidia-smi

Fri Jul 12 21:49:42 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 555.42.02              Driver Version: 555.42.02      CUDA Version: 12.5     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA RTX A6000               On  |   00000000:4F:00.0 Off |                  Off |
| 30%   31C    P0             87W /  300W |       2MiB /  49140MiB |     16%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
import transformers
import torch

In [3]:
# 원하는 경로에 다운 받았을 경우
model_id = "./Meta-Llama-3-8B-Instruct"

# 별도의 경로 지정 하지 않았을 경우 
# model_id = "meta-llama/Meta-Llama-3-8B-Instruct"

pipeline = transformers.pipeline(
  "text-generation",
  model=model_id,
  model_kwargs={"torch_dtype": torch.bfloat16},
  device="cuda",
)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


llama3는 현재 H100 80GB에서도 불가능 (140GB가 필요)

## Zero-shot Chain-of-Thought

### 1. Reasoning

\### instruction ###  
I want to make an AI pet profile with a picture of my dog.  
The given Input Query Text is the form of the AI pet profile that the user wants.  
So what do you think are the conditions for a good Prompt to go through a generative AI model and create a good result image?  

In [16]:
reasoning_prompt = [
    {"role": "system", 
     "content": 
     """You are a prompt engineer who makes prompts for generative AI models. """},
    
    {"role": "user", 
     "content": 
     f"""### instruction ###
      I want to make an AI pet profile with a picture of my dog. 
      The given Input Query Text is the form of the AI pet profile that the user wants. 
      So what do you think are the conditions for a good Prompt to go through a generative AI model and create a good result image?
      and If Input Query is Korean language, Please translate Korean to English.
      Please make sure the prompt includes the word 'dog' exactly four time. not others.
      Finally, Please ensure the generated prompt does not exceed the maximum token length allowed by the CLIP tokenizer.
      The maximum token length for the CLIP tokenizer is 60 tokens.
     """}
]

terminators = [
    pipeline.tokenizer.eos_token_id,
    pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

pipeline.model = pipeline.model.half().to(torch.float32)

reasoning_outputs = pipeline(
                    reasoning_prompt,
                    eos_token_id=terminators,
                    do_sample=True,
                    temperature=0.6,
                    top_p=0.9,
                )

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


In [17]:
conditions = reasoning_outputs[0]["generated_text"][-1]['content'];print(conditions)

As a prompt engineer, I'd suggest the following conditions for a good prompt to generate a high-quality AI pet profile image:

1. **Specificity**: The prompt should be specific about the desired output, including the type of image (pet profile), the subject (dog), and the desired style or theme.
2. **Clarity**: The prompt should be clear and concise, avoiding ambiguity and ensuring that the AI model understands the request.
3. **Relevance**: The prompt should be relevant to the desired output, including the image of the dog.
4. **Uniqueness**: The prompt should be unique and not easily duplicable, to encourage the AI model to generate a novel and creative output.
5. **Token length**: The prompt should not exceed the maximum token length allowed by the CLIP tokenizer (60 tokens).

Considering these conditions, here's a suggested prompt:

"Create a pet profile image featuring a dog, with a dog in the center, a dog's face in the top-right corner, and a dog's paw print in the bottom-left c

### 2. Question

- User Query 예시 (사용자에게 요청할 질문 : 어떤 스타일의 AI 프로필을 원하세요?)
1. A profile sitting next to a tree with a Christmas spirit.(크리스마스 분위기로 트리 옆에 앉아 있는 프로필)
2. Dog playing ball on the beach.(해변에서 공놀이하고 있는 강아지)
3. I want to make my pet under the desk with a lot of toys.(나는 내 펫이 책상 밑에서 많은 장난감들을 가지고 놀고 있도록 만들고싶어)
4. My dog is flying in the sky like an airplane.(내 강아지는 비행기와 같이 하늘을 날고 있어요.)
5. Please make it look like a profile picture of camping on the moon.(달에서 캠핑하고 있는 프로필 사진처럼 만들어줘.)
6. My dog is wearing glasses and sitting in the library reading a book.(내 강아지는 안경을 쓰고 도서관에 앉아 책을 읽고 있어)
7. A dog taking graduation photos!(졸업 사진 찍는 강아지!)
8. Please create a picture in the style of coding with a MacBook.(맥북으로 코딩하고 있는 스타일의 사진 부탁해)
9. A profile picture taken from above of being tossed in the air by everyone in the playground.(운동장에서 모두에게 헹가레를 받고 있는 모습을 위에서 찍은 프로필 사진)
10. Make it look like my dog is walking around wearing a helmet like a construction supervisor.(공사장 작업 간부처럼 헬멧을 쓰고 돌아다니는 분위기로 만들어줘)

샘플 입력 쿼리 10개. 더욱 일반적인 사용자의 입력값을 위해 스타일과 서술어를 다양하게 사용하였다.

Top-p (Nucleus Sampling):  
'Top-p' 샘플링은 모델이 특정 확률 값(p)에 따라 단어를 선택하는 방법입니다. 예를 들어, 'top-p 1'은 모델이 생성할 다음 단어를 선택할 때 가능한 모든 단어를 고려한다는 것을 의미합니다. 'p' 값이 1일 때, 모델은 가장 높은 확률의 단어를 선택하지 않고 전체 어휘에서 무작위로 단어를 선택합니다. 이는 텍스트 생성에서 다양성을 높이는 데 사용됩니다.

Temperature:  
'Temperature' 설정은 생성된 텍스트의 예측 가능성과 무작위성을 조절합니다. 'Temperature 1'은 모델이 상대적으로 균형 잡힌 방식으로 단어를 선택하도록 합니다. 값이 1보다 낮으면 모델은 더 확실한, 즉 확률이 높은 단어를 선택하는 경향이 있고, 값이 1보다 높으면 더 많은 무작위성을 가지고 단어를 선택합니다.

In [18]:
query_list = ['A profile sitting next to a tree with a Christmas mood.',
              'Dog playing ball on the beach.',
              'I want to make my pet under the desk with a lot of toys.',
              'My dog is flying in the sky like an airplane.',
              'Please make it look like a profile picture of camping on the moon.',
              'My dog is wearing glasses and sitting in the library reading a book.',
              'A dog taking graduation photos!',
              'Please create a picture in the style of coding with a MacBook.',
              'A profile picture taken from above of being tossed in the air by everyone in the playground.',
              'Make it look like my dog is walking around wearing a helmet like a construction supervisor.']

In [19]:
user_query = query_list[7];print(user_query)

Please create a picture in the style of coding with a MacBook.


In [35]:
user_query = '우리 강아지한테 크리스마스 코스튬을 입혀줘.'

In [20]:
prompt = [
    {"role": "system", 
     "content": 
     """You are a prompt engineer who makes prompts for generative AI models. 
        Your answer should always be prompt except explanation.
        All prompt factors should be separated by comma(like a, b, c)."""},
    
    {"role": "user", 
     "content": 
     f"""### instruction ###
    I want to make an AI pet profile with a picture of my dog. 
      The given Input Query Text is the form of the AI pet profile that the user wants. 
      So what do you think are the conditions for a good Prompt to go through a generative AI model and create a good result image?
      and If Input Query is Korean language, Please translate Korean to English.
      Please make sure the prompt includes the word 'dog' exactly four time. not others.
      Finally, Please ensure the generated prompt does not exceed the maximum token length allowed by the CLIP tokenizer.
      The maximum token length for the CLIP tokenizer is 60 tokens.
      ### Input Text ### 
      Query: {user_query} 
      Conditions: {conditions}
      
      =========
      New Prompt: 
     """}
]

terminators = [
    pipeline.tokenizer.eos_token_id,
    pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

outputs = pipeline(
    prompt,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.6,
    top_p=0.9,
)

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


In [21]:
answer = outputs[0]["generated_text"][-1]['content'];print(answer)

Create a pet profile image, featuring a dog, with a picture of my dog, showcasing my dog's adorable face, highlighting my dog's playful personality, and emphasizing my dog's loyalty.


In [11]:
str_answer = str(answer)
final_prompt = str_answer.replace('dog', '<s1> <s1>_1')
print(final_prompt)

Create a fun and festive AI pet profile for my <s1> <s1>_1, <s1> <s1>_1, sitting next to a Christmas tree with a joyful atmosphere, <s1> <s1>_1, surrounded by twinkling lights and festive decorations, with a caption that captures the holiday spirit and a dash of fun,
